# Part 1 - Import MNIST Data from TensorFlow

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

/home/achilles/anaconda3/envs/tfdeeplearning/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Helper Functions

In [4]:
def init_weights(shape):
    
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [5]:
def init_bias(shape):
    
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [6]:
def conv2d(x,W):
    
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [7]:
def max_pool_2by2(x):
    
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [8]:
def convolutional_layer(input_x,shape):
    
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [9]:
def normal_full_layer(input_layer,size):
    
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W)+b

# Part 2 - Create Placeholders and Layers

## Placeholders

In [10]:
x = tf.placeholder(tf.float32,shape=[None,784])

In [11]:
y_true = tf.placeholder(tf.float32,shape=[None,10])

## Layers

In [12]:
x_image = tf.reshape(x,[-1,28,28,1])

In [13]:
convo_1 = convolutional_layer(x_image,shape=[6,6,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [14]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[6,6,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [15]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [16]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [17]:
y_pred = normal_full_layer(full_one_dropout,10)

# Part 3 - Optimization and run Session

## Loss Function

In [18]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

## Optimizer

In [19]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)

In [20]:
train = optimizer.minimize(cross_entropy)

## Initialize Variables

In [21]:
init = tf.global_variables_initializer()

## Session

In [22]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')

Currently on step 0
Accuracy is:
0.1065


Currently on step 100
Accuracy is:
0.8601


Currently on step 200
Accuracy is:
0.9119


Currently on step 300
Accuracy is:
0.9308


Currently on step 400
Accuracy is:
0.937


Currently on step 500
Accuracy is:
0.9464


Currently on step 600
Accuracy is:
0.9486


Currently on step 700
Accuracy is:
0.9575


Currently on step 800
Accuracy is:
0.9595


Currently on step 900
Accuracy is:
0.9645


Currently on step 1000
Accuracy is:
0.964


Currently on step 1100
Accuracy is:
0.9663


Currently on step 1200
Accuracy is:
0.9658


Currently on step 1300
Accuracy is:
0.9716


Currently on step 1400
Accuracy is:
0.9725


Currently on step 1500
Accuracy is:
0.9728


Currently on step 1600
Accuracy is:
0.9748


Currently on step 1700
Accuracy is:
0.9744


Currently on step 1800
Accuracy is:
0.9742


Currently on step 1900
Accuracy is:
0.9776


Currently on step 2000
Accuracy is:
0.9775


Currently on step 2100
Accuracy is:
0.9763


Currently on step 2200
A

# Well Done